# Centros sociosanitarios y docentes

Se cargará y se preprocesará el dataset que contiene información básica sobre los centros sociosanitarios y docentes.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dbfread import DBF
import geopandas as gpd
import datetime
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Lectura del fichero
centros = gpd.read_file('../Data_input/Centros/Centros_Sociosanitarios_Docentes.shp')
centros.head()

,NOMBRE,TIPO,SUBTIPO,geometry
0,"NUEVA EDAD, CENTRO DE DIA",CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (444737.000 4476780.000)
1,EL ENCUENTRO,CENTROS DE DÍA,HOGARES Y CLUBES,POINT (444238.000 4475696.000)
2,"SANTA FILIPINA, RESIDENCIA DE PERSONAS MAYORES",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES,POINT (442684.000 4480112.000)
3,"RSDCIA. SACERDOTAL SAN PEDRO, RESIDENCIA DE PE...",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES MIXTA,POINT (440154.000 4475830.000)
4,"NTRA. SRA. DEL PILAR, RESIDENCIA DE PERSONAS M...",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES,POINT (448266.000 4477239.000)


In [3]:
print('Número de centros sociosanitarios y docentes:', len(centros))

Número de centros sociosanitarios y docentes: 2258


Se observa la cantidad de valores nulos y el recuento de centros por tipo y subtipo.

In [4]:
centros.isnull().sum()

NOMBRE         0
TIPO           0
SUBTIPO     1816
geometry       0
dtype: int64

In [5]:
centros.nunique()

NOMBRE      2257
TIPO           5
SUBTIPO        7
geometry    2090
dtype: int64

In [6]:
centros['TIPO'].value_counts()

TIPO
CENTROS DOCENTES             1750
CENTROS DE DÍA                273
CENTROS RESIDENCIALES         168
CENTROS SANITARIOS             66
CENTROS DE REHABILITACIÓN       1
Name: count, dtype: int64

In [7]:
centros['SUBTIPO'].value_counts()

SUBTIPO
CENTROS DE DÍA: OTROS                           161
HOGARES Y CLUBES                                112
RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES    106
RESIDENCIAS DE PERSONAS MAYORES MIXTA            52
CENTROS RESIDENCIALES: OTROS                      6
RESIDENCIAS DE PERSONAS MAYORES AUTONOMAS         4
CENTROS DE ATENCIÓN                               1
Name: count, dtype: int64

La columna que tiene la mayoría de valores nulos es `SUBTIPO`. Se va a eliminar, ya que no es necesaria y el tipo ya es definido en otra columna.

In [8]:
centros.drop('SUBTIPO', inplace=True, axis=1)

Además, en el atributo `TIPO`, hay un solo centro clasificado como "CENTRO DE REHABILITACIÓN". Para utilizar solo los centros principales ya que este aportaría poca información que pueda generalizarse, se descarta ese centro en concreto.

In [9]:
centro_rehab = centros[centros['TIPO']=='CENTROS DE REHABILITACIÓN']
centro_rehab

,NOMBRE,TIPO,geometry
327,"NEUROVIDA, CENTRO DE ATENCION",CENTROS DE REHABILITACIÓN,POINT (442480.000 4475279.000)


In [10]:
centros.drop(centro_rehab.index, axis=0, inplace=True)
centros.reset_index(drop=True, inplace=True)

In [11]:
centros.head()

,NOMBRE,TIPO,geometry
0,"NUEVA EDAD, CENTRO DE DIA",CENTROS DE DÍA,POINT (444737.000 4476780.000)
1,EL ENCUENTRO,CENTROS DE DÍA,POINT (444238.000 4475696.000)
2,"SANTA FILIPINA, RESIDENCIA DE PERSONAS MAYORES",CENTROS RESIDENCIALES,POINT (442684.000 4480112.000)
3,"RSDCIA. SACERDOTAL SAN PEDRO, RESIDENCIA DE PE...",CENTROS RESIDENCIALES,POINT (440154.000 4475830.000)
4,"NTRA. SRA. DEL PILAR, RESIDENCIA DE PERSONAS M...",CENTROS RESIDENCIALES,POINT (448266.000 4477239.000)


El atributo `geometry` se transforma en longitud y latitud para ver si las coordenadas son válidas.

In [12]:
centros['LATITUD'] = centros.geometry.x
centros['LONGITUD'] = centros.geometry.y

In [13]:
centros.loc[np.where((centros['LONGITUD']<0) | (centros['LATITUD']<0))]

,NOMBRE,TIPO,geometry,LATITUD,LONGITUD
132,CENTRO DE DIA FLORENCIA,CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
143,BALLESOL CENTRO RESIDENCIAL PASILLO VERDE,CENTROS RESIDENCIALES,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
159,"SANITAS RESIDENCIAL EL VISO, RESIDENCIA DE PER...",CENTROS RESIDENCIALES,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
201,"LAS VICTORIAS BEFEKO, CENTRO DE DIA",CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
214,"ARTURO PAJUELO, CENTRO MUNICIPAL DE MAYORES",CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
275,"NUESTRA SEÑORA DE LOS ANGELES, CENTRO MUNICIPA...",CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
303,"JUAN PABLO II AYUNTAMIENTO DE MADRID, CENTRO D...",CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
304,"PARQUE DE QUEROL, CENTRO DE DIA",CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
306,"SANTA ENGRACIA, CENTRO DE DIA PARA PERSONAS MA...",CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
318,"MANUEL ALEXANDRE AYUNTAMIENTO DE MADRID, CENTR...",CENTROS DE DÍA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308


In [14]:
geometries_errors = np.where((centros['LONGITUD']<0) | (centros['LATITUD']<0))[0]
geometries_errors

array([ 132,  143,  159,  201,  214,  275,  303,  304,  306,  318,  344,
        356,  357,  372,  385,  396,  406,  426,  596, 2137], dtype=int64)

Hay 20 puntos donde la latitud, longitud y, por tanto, la geometría, son valores infinitos. Por eso, se eliminan esos centros ya que no se dispone de su ubicación real.

In [15]:
centros_clean = centros.drop(index=geometries_errors).reset_index(drop=True)
centros_clean

,NOMBRE,TIPO,geometry,LATITUD,LONGITUD
0,"NUEVA EDAD, CENTRO DE DIA",CENTROS DE DÍA,POINT (444737.000 4476780.000),444737.0,4.476780e+06
1,EL ENCUENTRO,CENTROS DE DÍA,POINT (444238.000 4475696.000),444238.0,4.475696e+06
2,"SANTA FILIPINA, RESIDENCIA DE PERSONAS MAYORES",CENTROS RESIDENCIALES,POINT (442684.000 4480112.000),442684.0,4.480112e+06
3,"RSDCIA. SACERDOTAL SAN PEDRO, RESIDENCIA DE PE...",CENTROS RESIDENCIALES,POINT (440154.000 4475830.000),440154.0,4.475830e+06
4,"NTRA. SRA. DEL PILAR, RESIDENCIA DE PERSONAS M...",CENTROS RESIDENCIALES,POINT (448266.000 4477239.000),448266.0,4.477239e+06
...,...,...,...,...,...
2232,ESCUELA INFANTIL PEQUEANDO,CENTROS DOCENTES,POINT (444978.000 4482367.000),444978.0,4.482367e+06
2233,ESCUELA INFANTIL REGGIO,CENTROS DOCENTES,POINT (442867.000 4483980.000),442867.0,4.483980e+06
2234,ESCUELA INFANTIL TODO NIÑOS NURSERY SERRANO,CENTROS DOCENTES,POINT (442010.000 4477709.000),442010.0,4.477709e+06
2235,ESCUELA MUNICIPAL DE MÚSICA MORATALAZ,CENTROS DOCENTES,POINT (444196.000 4473679.000),444196.0,4.473679e+06


Finalmente, se eliminan las variables de longitud y latitud y se exporta el dataframe.

In [16]:
centros_clean.drop(['LATITUD', 'LONGITUD'], axis=1, inplace=True)
centros_clean.head()

,NOMBRE,TIPO,geometry
0,"NUEVA EDAD, CENTRO DE DIA",CENTROS DE DÍA,POINT (444737.000 4476780.000)
1,EL ENCUENTRO,CENTROS DE DÍA,POINT (444238.000 4475696.000)
2,"SANTA FILIPINA, RESIDENCIA DE PERSONAS MAYORES",CENTROS RESIDENCIALES,POINT (442684.000 4480112.000)
3,"RSDCIA. SACERDOTAL SAN PEDRO, RESIDENCIA DE PE...",CENTROS RESIDENCIALES,POINT (440154.000 4475830.000)
4,"NTRA. SRA. DEL PILAR, RESIDENCIA DE PERSONAS M...",CENTROS RESIDENCIALES,POINT (448266.000 4477239.000)


In [18]:
centros_clean_df = centros_clean.to_file('../Data_output/Centros/centros_sociosanitarios_docentes_clean.shp', index=False)

Con el fichero extraído, se cambia la geometría de los puntos por la de los edificios en QGIS, dejando el nombre, el tipo y el identificador del edificio. Ese fichero resultante será el que se utilice en Tableau para la creación de las visualizaciones correspondientes.